```{currentmodule} optimap
```

In [ ]:
# Code snippet for rendering animations in the docs
from IPython.display import HTML
import warnings
import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 2**128
import matplotlib.pyplot as plt

def render_ani_func(f):
    om.utils.disable_interactive_backend_switching()
    plt.switch_backend('Agg')
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        ani = f()
    %matplotlib inline
    om.utils.enable_interactive_backend_switching()

    vid = HTML(ani.to_html5_video())
    plt.close('all')
    return vid

```{tip}
Download this tutorial as a {download}`Jupyter notebook <converted/motion_compensation.ipynb>`, or a {download}`python script <converted/motion_compensation.py>` with code cells.
```

# Tutorial 4: Motion Compensation

This tutorial focuses on the motion tracking, motion-stabilization and motion artifact compensation capabilities of `optimap`.

```{warning}
This tutorial is currently in preparation. Please check back later for updates.
```
First, we import an example video file from our website [cardiacvision.ucsf.edu](https://cardiacvision.ucsf.edu) (this is done automatically using the code below). The video shows an action potential wave propagating across the ventricular surface of beating rabbit heart. The recording was peformed with voltage-sensitive fluorescent dye (Di-4-ANEPPS) and a high-speed camera (Basler acA720-520um) at 500fps. Experimenters: Jan Lebert, Namita Ravi & Jan Christoph (University of California, San Francisco, USA), 2022.

In [ ]:
import optimap as om
# import monochrome as mc  # remove this if you don't have monochrome installed

filename = om.utils.retrieve_example_data('Example_01_Sinus_Rabbit_Basler.npy')
video = om.load_video(filename)
video = om.video.rotate_left(video)

Alternatively, you can load your own video file by replacing `filename` with the name of your file (e.g. `filename = 'your_video_file.rsh'`), if it is located in the same folder as the script. If the video file is located somewhere else on your computer you can provide the path and filename (e.g. `'filename = /Users/userx/Desktop/your_video_file.rsh'`). Note, that we used {func}`optimap.video.rotate_left` to rotate the video to the left.

We can view this video using `optimap`'s built-in video player:

In [ ]:
om.video.play(video, title="original video with strong deformation");

In [ ]:
render_ani_func(lambda: om.video.play(video, 1, title="original video with strong deformation", interval=20))

You can see that the heart deforms strongly in our example video. Now, we are going to track the motion of the heart and will use the tracking data to create another motion-stabilized video in which the motion is absent:

In [ ]:
warped = om.motion.motion_compensate(video, 5, ref_frame=40)
flows_nocontrast = om.motion.estimate_displacements(video, 40)
warped_nocontrast = om.motion.warp_video(video, flows_nocontrast)
om.video.playn([video, warped, warped_nocontrast],
               titles=["original video", "with contrast-enhancement", "w/o contrast-enhancement"], figsize=(8, 3.5));

In [ ]:
warped_ref0 = om.motion_compensate(video, contrast_kernel=5, ref_frame=0)
warped_ref40 = om.motion_compensate(video, contrast_kernel=5, ref_frame=40)
om.video.playn([video, warped_ref40, warped_ref0], titles=["original video", "compensated ref 40", "compensated ref 0"], figsize=(8, 3.5));

In [ ]:
warped_ref0 = om.motion_compensate(video, contrast_kernel=5, ref_frame=0)
warped_ref40 = om.motion_compensate(video, contrast_kernel=5, ref_frame=40)
render_ani_func(lambda: om.video.playn([video, warped_ref40, warped_ref0], titles=["original video", "compensated ref 40", "compensated ref 0"], interval=20, figsize=(8, 3.5)))

In [ ]:
contrast3 = om.motion.contrast_enhancement(video[:300], 3)
contrast5 = om.motion.contrast_enhancement(video[:300], 5)
contrast9 = om.motion.contrast_enhancement(video[:300], 9)
om.video.playn([contrast3, contrast5, contrast9],
               titles=["contrast kernel 3", "contrast kernel 5", "contrast kernel 9"],
               skip_frame=3,
               figsize=(8, 3.5));

In [ ]:
def f():
    contrast3 = om.motion.contrast_enhancement(video[:300], 3)
    contrast5 = om.motion.contrast_enhancement(video[:300], 5)
    contrast9 = om.motion.contrast_enhancement(video[:300], 9)
    return om.video.playn([contrast3, contrast5, contrast9], titles=["contrast kernel 3", "contrast kernel 5", "contrast kernel 9"], skip_frame=1, figsize=(8, 3.5))
render_ani_func(f)